In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)

review_data_raw = pd.read_excel("review_data.xlsx")
review_data = review_data_raw[['brand', 'source_platform', 'source_type',
                              'date_time', 'comment_text_translated']]
review_data.rename(columns={'comment_text_translated':'review','date_time':'date',
                            'source_platform':'review_platform', 'source_type':'review_type'},inplace=True)
review_data['date'] = pd.to_datetime(review_data['date']).dt.date

review_data.sample(5)

,brand,review_platform,review_type,date,review
1611,Jupiler,Facebook,Social Media,2025-03-01,Union Sint-Gillis defeated Dender 4-1 on Saturday evening in the 28th matchday of the Jupiler Pro League at Dudenpark. The Unionists dominated the entire match.
217,Stella Artois,Twitter,Social Media,2025-07-23,@MarooninCar62 OMG and Stella Artois is a kind of Belgian beer hahahaha
4618,Jupiler,Tiktok,Social Media,2025-07-15,Squad’s favorite team [emoji_eyes][emoji_soccer_ball]️ #SportsTikTok #CercleBrugge #Football #JupilerProLeague
10036,Jupiler,Instagram,Social Media,2025-06-18,@margo.lkr [emoji_raising_hands][emoji_raising_hands]
10554,Jupiler,Instagram,Social Media,2025-06-27,dinguerieeee I hope bcppp @elina.demona [emoji_fire]


## Review preprocessing

In [3]:
import re
from langdetect import detect_langs, DetectorFactory

corona_review = review_data[review_data['brand']=='Corona'].drop_duplicates().dropna().reset_index(drop=True)

"""
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Clean the reviews
"""
def clean_text(text):
    if pd.isna(text):
        return ""
    
    # 1. Remove emoji placeholders like [emoji_tropical_drink]
    text = re.sub(r"\[emoji_[^\]]+\]", "", text)
    # 2. Lowercase
    text = text.lower() 
    # 3. Remove HTML tags
    text = re.sub(r"<.*?>", "", text)
    # 4. Keep only letters, numbers, basic punctuation
    text = re.sub(r"[^a-zA-Z0-9\s.,!?\$€£₹%/-]", "", text)
    # 5. Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()
    
    return text


corona_review['review_cleaned'] = corona_review['review'].apply(clean_text)


"""
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Check if the review is in english or not
"""

DetectorFactory.seed = 0  # for consistent results

def is_english(text):
    try:
        langs = detect_langs(text)
        # get highest probability
        top = max(langs, key=lambda x: x.prob)
        return top.lang == 'en' and top.prob > 0.90
    except:
        return False

corona_review['is_english'] = corona_review['review_cleaned'].apply(is_english)


"""
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Translate non English reviews to English
"""
from deep_translator import GoogleTranslator

def translate_to_english(text):
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except:
        return text  # fallback: return original
    
corona_review.loc[corona_review['is_english'] == False,'review_cleaned'] = corona_review.loc[corona_review['is_english'] == False,'review_cleaned'].apply(translate_to_english)


"""
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Extract emojis from the reviews
"""

def extract_emojis(text):
    if pd.isna(text):
        return "No Emojis"
    
    emojis = re.findall(r"\[emoji_[^\]]+\]", text)
    
    if len(emojis) == 0:
        return "No Emojis"
    
    return emojis

corona_review['emojis'] = corona_review['review'].apply(extract_emojis)


"""
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Extract hashtags from the reviews
"""
def extract_hashtags(text: str):
    if pd.isna(text):
        return "No Hashtags"
    
    Hashtags = re.findall(r"#\w+", text)
    
    if len(Hashtags) == 0:
        return "No Hashtags"
    
    return Hashtags

corona_review['hashtags'] = corona_review['review'].apply(extract_hashtags)

corona_review.sample(5)



,brand,review_platform,review_type,date,review,review_cleaned,is_english,emojis,hashtags
87,Corona,Twitter,Social Media,2025-02-27,@GevaertThomas My respect for the memory of the Belgian voter is lower. It's still a long way until the next elections. If you think about what happened during Corona and that certain parties that were in charge at that time have gained electorally.,gevaertthomas my respect for the memory of the belgian voter is lower. its still a long way until the next elections. if you think about what happened during corona and that certain parties that were in charge at that time have gained electorally.,True,No Emojis,No Hashtags
30,Corona,Facebook,Social Media,2025-03-10,Anyone interested in taking over Corona beer? For €25 it can go. The first one to respond gets it! Cheers,anyone interested in taking over corona beer? for €25 it can go. the first one to respond gets it! cheers,True,No Emojis,No Hashtags
198,Corona,Instagram,Social Media,2025-08-14,"Find me at the beach this Friday to celebrate 100 years @corona with @vilebrequin [emoji_shorts]\n\n[emoji_round_pushpin] Riverwoods Knokke\n[emoji_eight-thirty] 2pm \n\nThis is an open event, everybody is welcome and hope to see you there [emoji_sun_with_face][emoji_umbrella_on_ground]\n\n#advertising #corona100 #thisisliving","find me at the beach this friday to celebrate 100 years corona with vilebrequin riverwoods knokke 2pm this is an open event, everybody is welcome and hope to see you there advertising corona100 thisisliving",True,"[[emoji_shorts], [emoji_round_pushpin], [emoji_eight-thirty], [emoji_sun_with_face], [emoji_umbrella_on_ground]]","[#advertising, #corona100, #thisisliving]"
178,Corona,Facebook,Social Media,2025-05-20,"[emoji_fire] 𝗙𝗜𝗡𝗔𝗟𝗘 𝗘𝗨𝗥𝗢𝗣𝗔 𝗟𝗘𝗔𝗚𝗨𝗘 [emoji_soccer_ball]️ \n\n[emoji_fire] 𝗖𝗘 𝗠𝗘𝗥𝗖𝗥𝗘𝗗𝗜 𝟮𝟭/𝟬𝟱 , 𝟮𝟭𝗛𝟬𝟬\n\n[emoji_right_arrow] Big screen, atmosphere, live commentary, and special promotions during the match!\n\n[emoji_right_arrow] 𝗣𝗥𝗢𝗠𝗢 𝗗𝗨𝗥𝗔𝗡𝗧 𝗟𝗘 𝗠𝗔𝗧𝗖𝗛 [emoji_partying_face]\n\n[emoji_backhand_index_pointing_right] 3 DESPÉ purchased = 1 free\n[emoji_backhand_index_pointing_right] 3 CORONA purchased = 1 free *\n[emoji_backhand_index_pointing_right] 1 METER of beer = 25€ *\n* also available alcohol-free\n\n#Bowling #bowlingthemis[emoji_bowling] #promo #europaleague #EuropaLeague #foot","/ , big screen, atmosphere, live commentary, and special promotions during the match! 3 desp purchased 1 free 3 corona purchased 1 free 1 meter of beer 25€ also available alcohol-free bowling bowlingthemis promo europaleague europaleague foot",True,"[[emoji_fire], [emoji_soccer_ball], [emoji_fire], [emoji_right_arrow], [emoji_right_arrow], [emoji_partying_face], [emoji_backhand_index_pointing_right], [emoji_backhand_index_pointing_right], [emoji_backhand_index_pointing_right], [emoji_bowling]]","[#Bowling, #bowlingthemis, #promo, #europaleague, #EuropaLeague, #foot]"
249,Corona,Twitter,Social Media,2025-10-06,"McLaren CEO Zak Brown and team principal Andrea Stella shared their elation after Sunday’s Constructors’ coronation in Singapore, praising the team’s relentless spirit and setting their sights on a bright future. https://t.co/XJhp8a5vSQ","mclaren ceo zak brown and team principal andrea stella shared their elation after sundays constructors coronation in singapore, praising the teams relentless spirit and setting their sights on a bright future. https//t.co/xjhp8a5vsq",True,No Emojis,No Hashtags


## Check the relevance of the reviews

In [4]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")
ref_emb = model.encode("This is a review about beer", convert_to_tensor=True)

beer_keywords = ["beer", "brew", "lager", "ipa", "stout", "pils", "ale", "corona", 
                 "heineken", "budweiser", "jupiler", "leffe","craft beer", "bottle", "keg",
                 'taste', 'lime', 'lemon', 'brewery']

exclude_keywords = ["covid", "covid19", "covid-19", "coronavirus", "corona virus",
                    "pandemic", "outbreak", "lockdown", "quarantine", "isolation",
                    "social distancing", "stay at home", "covid test", "tested positive",
                    "positive test", "negative test", "antigen", "pcr", "rapid test",
                    "epidemic", "virus", "viral infection"]

def is_beer_related(text, threshold=0.45):
    
    # Rule-based exclusion
    if any(kword in text for kword in exclude_keywords):
        return False

    # Rule-based inclusion
    if any(kword in text for kword in beer_keywords):
        return True
    
    # Semantic fallback
    emb = model.encode(text, convert_to_tensor=True)
    score = util.cos_sim(emb, ref_emb).item()
    return score >= threshold


corona_review["is_beer_related"] = corona_review["review_cleaned"].apply(is_beer_related)

corona_review.sample(5)

,brand,review_platform,review_type,date,review,review_cleaned,is_english,emojis,hashtags,is_beer_related
188,Corona,Twitter,Social Media,2025-08-08,"@Bob4Florida @quetzalponk @lang_domin97000 @BoxCarLabs @dinosaurs1969 Corona is not German style because it contains corn. The same goes for Stouts that contain adjuncts. German style contains only water, barley, hops & yeast, nothing more or it isn't German style.\n\nTrappist is not a style, it contains multiple. It only indicates that it is brewed by monks.","bob4florida quetzalponk langdomin97000 boxcarlabs dinosaurs1969 corona is not german style because it contains corn. the same goes for stouts that contain adjuncts. german style contains only water, barley, hops yeast, nothing more or it isnt german style. trappist is not a style, it contains multiple. it only indicates that it is brewed by monks.",True,No Emojis,No Hashtags,True
235,Corona,Untappd,Online Review,2025-08-27,Padel time,fell with it,False,No Emojis,No Hashtags,False
166,Corona,Facebook,Social Media,2025-05-30,[emoji_clinking_beer_mugs] Corona makes you dream of sunny beaches and brings the sun closer with every sip! Let yourself be tempted by a refreshing Corona Extra or alcohol-free Corona Cero 0.0% with your favorite lunch and taste the sun in every sip.\n\nUntil June 29 for a summer-soft promotional price[emoji_sun],corona makes you dream of sunny beaches and brings the sun closer with every sip! let yourself be tempted by a refreshing corona extra or alcohol-free corona cero 0.0% with your favorite lunch and taste the sun in every sip. until june 29 for a summer-soft promotional price,True,"[[emoji_clinking_beer_mugs], [emoji_sun]]",No Hashtags,True
107,Corona,Twitter,Social Media,2025-05-07,Bravo [emoji_clapping_hands][emoji_clapping_hands][emoji_clapping_hands] Discover the video of KORONA.GLIWICE! #TikTok https://t.co/1itkupm2bu,bravo discover the video of korona.gliwice! tiktok https//t.co/1itkupm2bu,True,"[[emoji_clapping_hands], [emoji_clapping_hands], [emoji_clapping_hands]]",[#TikTok],False
157,Corona,Facebook,Social Media,2025-07-30,Corona action and Corona Zero in our drinks [emoji_sun][emoji_clinking_beer_mugs]\n\nTry to win a Corona cooler! [emoji_beach_with_umbrella]\n\n[emoji_shopping_bags] Action available in our Belgian stores: https://www.maziers.com/shops/marche-en-famenne-brasserie-maziers/,corona action and corona zero in our drinks try to win a corona cooler! action available in our belgian stores https//www.maziers.com/shops/marche-en-famenne-brasserie-maziers/,True,"[[emoji_sun], [emoji_clinking_beer_mugs], [emoji_beach_with_umbrella], [emoji_shopping_bags]]",No Hashtags,True


In [5]:
# pd.set_option('display.colheader_justify', 'left')
# corona_review[corona_review['is_beer_related'] == False]['review_cleaned']
corona_review['is_beer_related'].value_counts()

is_beer_related
True     260
False     35
Name: count, dtype: int64

### Remove Non-Beer related reviews

In [6]:
corona_review = corona_review[corona_review['is_beer_related'] == True].reset_index(drop=True)

corona_review.sample(5)

,brand,review_platform,review_type,date,review,review_cleaned,is_english,emojis,hashtags,is_beer_related
15,Corona,Twitter,Social Media,2025-02-02,"A few years ago, we all got corona as a family, but of course, we didn't know we had corona. My dad quickly took me and my brother to the hospital in the morning, and without looking at anything, they said it was an infection. They put this yellow serum on me, and at that moment, my brother was also getting serum next to me.","a few years ago, we all got corona as a family, but of course, we didnt know we had corona. my dad quickly took me and my brother to the hospital in the morning, and without looking at anything, they said it was an infection. they put this yellow serum on me, and at that moment, my brother was also getting serum next to me.",True,No Emojis,No Hashtags,True
17,Corona,Facebook,Social Media,2025-01-14,Corona Cero 100% Refunded*,corona zero 100% refunded,False,No Emojis,No Hashtags,True
224,Corona,Instagram,Social Media,2025-10-10,"How about ending the day like this? \n\nTag that person who will keep you company during this happy hour. \n\nIn addition to an exclusive menu with shareable snacks, Happy Quotidien offers double glasses of Aperol, wines, and sparkling wine, or if you order a bucket with five long necks (Corona and Stella Artois), Friday is on us! \n\n#HappyQuotidien #LePainQuotidienBR #MomentoQuotidien","how about ending the day like this? tag that person who will keep you company during this happy hour. in addition to an exclusive menu with shareable snacks, happy quotidien offers double glasses of aperol, wines, and sparkling wine, or if you order a bucket with five long necks corona and stella artois, friday is on us! happyquotidien lepainquotidienbr momentoquotidien",True,No Emojis,"[#HappyQuotidien, #LePainQuotidienBR, #MomentoQuotidien]",True
34,Corona,Facebook,Social Media,2025-05-14,"After the Bierstal closed its doors shortly after corona, the Bosstraat neighborhood has been 'without beer' for quite some time.","after the bierstal closed its doors shortly after corona, the bosstraat neighborhood has been without beer for quite some time.",True,No Emojis,No Hashtags,True
238,Corona,Twitter,Social Media,2025-09-02,"@ezeluc Corona, war in Ukraine, everything is propaganda. Who writes all this nonsense? Russian Pravda was small beer compared to this constant stream of crap.","ezeluc corona, war in ukraine, everything is propaganda. who writes all this nonsense? russian pravda was small beer compared to this constant stream of crap.",True,No Emojis,No Hashtags,True


## Stopwords removal

In [7]:
import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')

def clean_stopwords(text):
    # Base NLTK stopword list
    base_sw = set(stopwords.words("english"))
    
    # Words we want to keep for sentiment & topics
    keep_words = {
        "not", "no", "never",          # negation is important
        "very", "too", "more",         # intensity important in sentiment
        "good", "bad", "great", "amazing", "terrible",  # sentiment words
        "beer", "ipa", "lager", "stout", "ale", "pils", "corona"  # domain words
    }
    
    # Remove kept words from stopwords
    base_sw = base_sw - keep_words

    # Add domain-specific stopwords
    domain_sw = {
        "drink", "drinking", "drank", "alcohol",
        "brew", "brewing", "brewed",
        "bottle", "can", "canned", "glass", "pint",
        "episode", "review", "reviews",
        "beerlovers", "beerlover"
    }
    
    final_stopwords = base_sw.union(domain_sw)
    
    # Tokenize
    words = re.findall(r"\b\w+\b", text.lower())
    
    # Remove stopwords
    filtered = [w for w in words if w not in final_stopwords]
    
    return " ".join(filtered)

corona_review["review_cleaned"] = corona_review["review_cleaned"].apply(clean_stopwords)

corona_review.sample(5)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhij\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,brand,review_platform,review_type,date,review,review_cleaned,is_english,emojis,hashtags,is_beer_related
57,Corona,Facebook,Social Media,2025-07-14,"[emoji_sun]Summer aperitifs are now even better! [emoji_sun] From July 14 to 27, you can enjoy 1+1 on Lays Max, Jupiler, and Corona at your favorite Carrefour store via Deliveroo! Don't miss this great deal and order quickly!",summer aperitifs even better july 14 27 enjoy 11 lays max jupiler corona favorite carrefour store via deliveroo dont miss great deal order quickly,True,"[[emoji_sun], [emoji_sun]]",No Hashtags,True
58,Corona,Facebook,Social Media,2025-05-24,"[emoji_beer_mug] Corona... on draft in the Latin Quarter! Ultra fresh, light, and refreshing, the famous Mexican beer tastes even better on tap! [emoji_sun_with_face] A splash of lime, a chill vibe, and it's summer in your glass. Come enjoy it now! [emoji_lemon]",corona draft latin quarter ultra fresh light refreshing famous mexican beer tastes even better tap splash lime chill vibe summer come enjoy,True,"[[emoji_beer_mug], [emoji_sun_with_face], [emoji_lemon]]",No Hashtags,True
140,Corona,Facebook,Social Media,2025-07-09,[emoji_warning] Don't forget the current promotions and actions [emoji_warning]\n\n[emoji_pushpin] On all rosés: 5+1 FREE all summer!!\n[emoji_pushpin] Chimay spirit: TRY TO WIN A SUN LOUNGER OR A MAGNUM (drawing on July 19)\n[emoji_pushpin] Corona: TRY TO WIN 2 TICKETS FOR DOUR FESTIVAL [emoji_exploding_head] (scratch ticket)\n\nGO GO GO,dont forget current promotions actions ross 51 free summer chimay spirit try win sun lounger magnum drawing july 19 corona try win 2 tickets dour festival scratch ticket go go go,True,"[[emoji_warning], [emoji_warning], [emoji_pushpin], [emoji_pushpin], [emoji_pushpin], [emoji_exploding_head]]",No Hashtags,True
70,Corona,Facebook,Social Media,2025-03-19,"A 38-YEAR-OLD MAN WAS SENTENCED TO PAY 60,000 EUROS IN DAMAGES FOR BEATING A POLICE OFFICER IN GENT. The 38-year-old man living in Lochristi was involved in a violent incident on December 26, 2021, during the corona period. The man beat a police officer in Gent. He punched and kicked the police officer. In his statement in court, the police officer said, 'My mother had to wash me, had to dress me. I couldn't do anything.' These words reveal the extent of his victimization. The injured police officer stated that he suffered significant physical and emotional harm.",38 year old man sentenced pay 60 000 euros damages beating police officer gent 38 year old man living lochristi involved violent incident december 26 2021 corona period man beat police officer gent punched kicked police officer statement court police officer said mother wash dress couldnt anything words reveal extent victimization injured police officer stated suffered significant physical emotional harm,True,No Emojis,No Hashtags,True
33,Corona,Instagram,Social Media,2025-07-13,Lime [emoji_lime] is so 90s. Long live the jalapeño!!! #coronabeer #mexico #jalapegno #hotbeer #SummerDrinks,lime 90s long live jalapeo coronabeer mexico jalapegno hotbeer summerdrinks,True,[[emoji_lime]],"[#coronabeer, #mexico, #jalapegno, #hotbeer, #SummerDrinks]",True


In [10]:
corona_review.to_excel("Corona Reviews Cleaned.xlsx",index=False)